In [0]:
import numpy as np
import heapq as hq
from itertools import count

Every Node object represent a node on the search tree.

It stores
1.   The state: position of all the tiles
2.   A reference to the parent node
3.   g: cost of reaching the node (considering cost of each edge in the tree as 1)
4.   f-value and
5.   a: represents the action applied on parent node to reach this node

and has two funcitons:

1.   h()
2.   expand()

In [0]:
class Node:
  def __init__(self, state, parent, cost, action):
    self.state = state
    self.parent = parent
    self.g = cost
    self.f = self.g
    self.a = action # Action applied to reach the state L, R, T or D

  # Heuristic: sum of manhattan distances of each tile.. blank included
  def h(self, goal):
    sum = 0
    for i in range(3):
      for j in range(3):
        m, n = np.where(goal == self.state[i][j]) # np.where(a == b) returns the array of positions in the array a, where the element is == b
        m, n = int(m), int(n)
        sum += abs(i - m) + abs(j - n)
    return sum

  # Moves the position of the blank and returns an array (a list) of Nodes obtained
  def expand(self):
    i, j = np.where(self.state == 0)
    i, j = int(i), int(j) # Current position of the blank -> {0}
    children = []

    # Move the blank one step at a time
    # Does not enter parent state again by checking 'Node.a' values
    # Actions: D, R, L and T denote the movement of 0 in 4 directions

    if self.a != 'D' and i - 1 >= 0:
        s = self.state.copy()
        s[i][j] = s[i - 1][j]
        s[i - 1][j] = 0
        children.append(Node(s, self, self.g + 1, 'T')) 

    if self.a != 'R' and j - 1 >= 0:
        s = self.state.copy()
        s[i][j] = s[i][j - 1]
        s[i][j - 1] = 0
        children.append(Node(s, self, self.g + 1, 'L'))
    
    if self.a != 'L' and j + 1 <= 2:
        s = self.state.copy()
        s[i][j] = s[i][j + 1]
        s[i][j + 1] = 0
        children.append(Node(s, self, self.g + 1, 'R'))

    if self.a != 'T' and i + 1 <= 2:
        s = self.state.copy()
        s[i][j] = s[i + 1][j]
        s[i + 1][j] = 0
        children.append(Node(s, self, self.g + 1, 'D'))
    return children

Recursive function to print the path to solution

In [0]:
def print_soln(sol):
  if sol.parent == None:
    print(sol.state, '\n')
    return
  print_soln(sol.parent)
  print(sol.state, "\n")

**Recursive Best First Search**

Used a *heap* to store the Nodes in every depth-level, works as a priority queue


In [0]:
def RBFS(node, goal, f_limit):
  if np.allclose(goal, node.state): # np.allclose checks if both the arrays are identical
    print("Solution found")
    print_soln(node)
    return ("cost:", node.f)
  
  scsrs = node.expand()
  if len(scsrs) == 0:
    return ('failure', float('inf'))
  pq = []
  u = count() # Not very important count..

  for s in scsrs:
    s.f = max(s.h(goal) + s.g, node.f)
    hq.heappush(pq, (s.f, next(u), s)) # Stores Nodes with priority based on the f-value, 'u' is an unique integer to break ties
  
  while True:
    f_best, u1, best = hq.heappop(pq) # Node with least f-value
    if best.f > f_limit:
      return ('failure', best.f)

    try:
      f_alt, u2, alt = hq.heappop(pq) # Second least f-value
      hq.heappush(pq, (f_alt, u2, alt))
    except IndexError: # Error is raised if there isn't an alternative node.
      f_alt = f_limit

    result, best.f = RBFS(best, goal, min(f_limit, f_alt))
    hq.heappush(pq, (best.f, u1, best)) # Updates the values as the recurion unwinds

    if result != 'failure':
      return (result, best.f)

All the states are represented with a 3x3 array

0 represents the blank tile

Tile movements are implimented by swapping 0 with adjacent elements

In [12]:
puzzle = np.asarray([[7, 0, 4], [5, 2, 6], [8, 3, 1]])

goal = list(np.arange(1, 9))
goal.append(0)
goal = np.asarray(goal).reshape(3, 3)

start = Node(puzzle, None, 0, '')
RBFS(start, goal, float('inf'))

Solution found
[[7 0 4]
 [5 2 6]
 [8 3 1]] 

[[7 2 4]
 [5 0 6]
 [8 3 1]] 

[[7 2 4]
 [5 3 6]
 [8 0 1]] 

[[7 2 4]
 [5 3 6]
 [8 1 0]] 

[[7 2 4]
 [5 3 0]
 [8 1 6]] 

[[7 2 4]
 [5 0 3]
 [8 1 6]] 

[[7 2 4]
 [0 5 3]
 [8 1 6]] 

[[0 2 4]
 [7 5 3]
 [8 1 6]] 

[[2 0 4]
 [7 5 3]
 [8 1 6]] 

[[2 4 0]
 [7 5 3]
 [8 1 6]] 

[[2 4 3]
 [7 5 0]
 [8 1 6]] 

[[2 4 3]
 [7 0 5]
 [8 1 6]] 

[[2 4 3]
 [7 1 5]
 [8 0 6]] 

[[2 4 3]
 [7 1 5]
 [0 8 6]] 

[[2 4 3]
 [0 1 5]
 [7 8 6]] 

[[2 4 3]
 [1 0 5]
 [7 8 6]] 

[[2 0 3]
 [1 4 5]
 [7 8 6]] 

[[0 2 3]
 [1 4 5]
 [7 8 6]] 

[[1 2 3]
 [0 4 5]
 [7 8 6]] 

[[1 2 3]
 [4 0 5]
 [7 8 6]] 

[[1 2 3]
 [4 5 0]
 [7 8 6]] 

[[1 2 3]
 [4 5 6]
 [7 8 0]] 



('cost:', 25)